# AI Comic Book Generator - Colab Launcher

This notebook sets up the environment and runs the AI Comic Generator pipeline.
It uses your GitHub repository as the source of truth.

In [ ]:
# @title 1. Setup Environment
# @markdown Enter your GitHub repo URL and branch.
REPO_URL = "https://github.com/YourUser/comic-gen.git" # @param {type:"string"}
BRANCH = "main" # @param {type:"string"}

import os

if not os.path.exists("comic-gen"):
    !git clone -b $BRANCH $REPO_URL comic-gen
else:
    %cd comic-gen
    !git pull
    %cd ..

%cd comic-gen
!pip install -r requirements.txt
print("Environment Set Up!")

In [ ]:
# @title 2. Configuration
# @markdown Enter API Keys (Secrets are recommended, but input here for quickstart)
import os
from google.colab import userdata

# Try getting from Colab Secrets first
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
except:
    pass

# Manual override if needed
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("Enter OpenAI API Key: ")
if not os.environ.get("HF_TOKEN"):
    os.environ["HF_TOKEN"] = input("Enter Hugging Face Token: ")

In [ ]:
# @title 3. Run Pipeline
# @markdown Upload your story file to the 'comic-gen' folder file browser on the left.
INPUT_FILE = "story.txt" # @param {type:"string"}
OUTPUT_DIR = "output" # @param {type:"string"}
HF_REPO_ID = "your-username/comic-dataset" # @param {type:"string"}

# Run the main script
!python src/main.py --input $INPUT_FILE --output $OUTPUT_DIR --colab --storage hf --hf_repo $HF_REPO_ID

In [ ]:
# @title 4. Download Output (Optional)
# @markdown If you didn't use HF storage, download the zip here.
from google.colab import files
!zip -r comic_output.zip $OUTPUT_DIR
files.download('comic_output.zip')